## import libraries

In [1]:
from compas.geometry import Point, Plane, Frame, Circle, Cylinder
from compas_vol.microstructures import LatticePolar
from compas_vol.primitives import VolCylinder
from compas_vol.combinations import SmoothUnion, Intersection

In [2]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes
from compas_vol.utilities import bbox_edges

## create volumetric object (CSG tree)

In [3]:
lat = LatticePolar(1, 8.0, 1.2, 8, frame=Frame([0,0,0],[0,0,1],[1,0,0]))

In [4]:
c1 = VolCylinder(Cylinder(Circle(Plane([0,0,0], [0,1,0]), 1.0), 39))
c2 = VolCylinder(Cylinder(Circle(Plane([0,0,0], [0,1,0]), 12.0), 36))

In [5]:
i = Intersection(c2, lat)
su = SmoothUnion(c1, i, 2.5)

## workspace (dense grid)

In [6]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -16.0, 16.0
lby, uby = -20.0, 20.0
lbz, ubz = -16.0, 16.0
# resolution(s)
nx, ny, nz = 124, 184, 124
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

## sample at discrete interval

In [7]:
m = su.get_distance_numpy(x, y, z)

## generate isosurface (marching cube)

In [8]:
v, f, n, l = marching_cubes(m, 0, spacing=(gx, gy, gz))
v += [lbx,lby,lbz]

## display mesh

In [9]:
print('use fc slider for cutting plane')
p = mp.plot(v, f, c=np.array([0,0.57,1.0]), shading={"flat":False, "roughness":0.4, "metalness":0.01, "reflectivity":1.0})
vs,ve = bbox_edges(lbx,ubx,lby,uby,lbz,ubz)
p.add_lines(np.array(vs), np.array(ve))

@mp.interact(fc=(0,420000))
def facefilter(fc=420000):
    p.update_object(faces=f[:fc])

use fc slider for cutting plane


/Users/bernham/anaconda3/envs/compasdev/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.129032…

interactive(children=(IntSlider(value=420000, description='fc', max=420000), Output()), _dom_classes=('widget-…